In [2]:
import sys
sys.path.append(r'C:\Users\damod\OneDrive\Área de Trabalho\Projetos Python\dataprocessing')

Importação das bibliotecas

In [3]:
import dataprocessing.datacolect as dc
import re
import pandas as pd
import numpy as np

# 1. Sobre coleta dos dados na Wikipedia

# 2. Rotina de ***scraping***

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_Xbox_Series_X_and_Series_S_games'

In [5]:
soup_class = dc.Soup(url)
soup = soup_class.get_soup()

In [6]:
scrape = dc.Scrape()

In [10]:
games_table = scrape.find_elements(
    tag='tbody', soup=soup,
)
games_table = games_table[1]

In [17]:
games_data = scrape.find_elements(
    tag='tr', soup=games_table,
)
games_data = games_data[2:]

In [64]:
def scraping_games(games: list, init_info: int=1, last_info: int=3):
    """
    Realiza o scraping das informações dos jogos presentes na tabela da lista da Wikipedia.
    """
    scraped_game = dict()
    scraped_games = list()

    for game in games:
        try: 
            name_soup = scrape.find_element(tag='th', soup=game)
            name = re.sub(r'\n', '', name_soup.get_text())
            scraped_game['name'] = name

            another_infos = scrape.find_elements(tag='td', soup=game)
            games_info = list()

            for another_info in another_infos[init_info:last_info]:
                info = re.sub(r'\n', '', another_info.get_text())
                games_info.append(info)
            scraped_game['infos'] = games_info
            scraped_games.append(scraped_game.copy())
        except AttributeError:
            name_soup = scrape.find_element(tag='td', soup=game)
            name = re.sub(r'\n', '', name_soup.get_text())
            scraped_game['name'] = name

            for another_info in another_infos[init_info:last_info]:
                info = re.sub(r'\n', '', another_info.get_text())
                games_info.append(info)
            scraped_game['infos'] = games_info
            scraped_games.append(scraped_game.copy())
    return scraped_games


In [65]:
wiki_xbox_series = pd.DataFrame(scraping_games(games_data))

<td><link href="mw-data:TemplateStyles:r1129693374" rel="mw-deduplicated-inline-style"/><div class="hlist"><ul><li><a href="/wiki/343_Industries" title="343 Industries">343 Industries</a></li><li><a href="/wiki/Bungie" title="Bungie">Bungie</a></li></ul></div>
</td>
<td><a href="/wiki/Xbox_Game_Studios" title="Xbox Game Studios">Xbox Game Studios</a>
</td>


In [55]:
wiki_xbox_series['developer'] = wiki_xbox_series['infos'].apply(lambda x: x[0])
wiki_xbox_series['publisher'] = wiki_xbox_series['infos'].apply(lambda x: x[1])
wiki_xbox_series = wiki_xbox_series.drop(columns='infos')

In [57]:
wiki_xbox_series.loc[wiki_xbox_series['name'].str.contains('Halo')]

,name,developer,publisher
175,Halo Infinite,343 Industries,Xbox Game Studios
176,Halo: The Master Chief Collection,343 IndustriesBungie,Xbox Game Studios
